## Set Up

In [ ]:
import pandas as pd

In [61]:
# Create function to import multiple data files

data_path = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_"

def import_data(date):
    """
    import data
    clean column names
    """
    dfname = pd.read_csv(data_path+date+".txt")
    dfname.columns = dfname.columns.str.strip()
    return dfname
          
mta190504_raw = import_data("190504")
mta190511_raw = import_data("190511")
mta190518_raw = import_data("190518")
mta190525_raw = import_data("190525")
mta190601_raw = import_data("190601")


In [62]:
# Concatenate all data
frames = [mta190504_raw, mta190511_raw, mta190518_raw, mta190525_raw, mta190601_raw]

mta19_raw = pd.concat(frames).reset_index().drop(["index"], axis=1)
mta19_raw.head(5)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,00:00:00,REGULAR,7035249,2384833
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,04:00:00,REGULAR,7035269,2384840
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,08:00:00,REGULAR,7035292,2384875
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,12:00:00,REGULAR,7035392,2384951
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,16:00:00,REGULAR,7035651,2385020


In [ ]:
# Add and format new variables

def data_clean(dfname,dfname2):
    """
    add and format new variables
    """
    return dfname2

data_clean(mta19_raw, mta19)
#etc. - can run it for multiple years

In [ ]:
# Function to output grouped data

def grouped_data(dfname2):
    """
    add statements to return grouped dataframes
    """
    return grouped_data


## Below is scratchwork

In [63]:
mta_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822989 entries, 0 to 822988
Data columns (total 11 columns):
C/A         822989 non-null object
UNIT        822989 non-null object
SCP         822989 non-null object
STATION     822989 non-null object
LINENAME    822989 non-null object
DIVISION    822989 non-null object
DATE        822989 non-null object
TIME        822989 non-null object
DESC        822989 non-null object
ENTRIES     822989 non-null int64
EXITS       822989 non-null int64
dtypes: int64(2), object(9)
memory usage: 69.1+ MB


In [64]:
mta_raw.sort_values(["STATION","LINENAME","DIVISION","C/A","UNIT","SCP","DATE","TIME","DESC"])
mta_raw.head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,00:00:00,REGULAR,7035249,2384833
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,04:00:00,REGULAR,7035269,2384840
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,08:00:00,REGULAR,7035292,2384875
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,12:00:00,REGULAR,7035392,2384951
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,16:00:00,REGULAR,7035651,2385020
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/27/2019,20:00:00,REGULAR,7035930,2385070
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2019,00:00:00,REGULAR,7036100,2385087
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2019,04:00:00,REGULAR,7036119,2385088
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2019,08:00:00,REGULAR,7036125,2385103
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,04/28/2019,12:00:00,REGULAR,7036197,2385155


In [65]:
mta1 = mta_raw.copy()
mta1["time_hour"] = pd.to_numeric(mta1["TIME"].str[0:2])
mta1.tail(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,time_hour
822979,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/23/2019,09:00:00,REGULAR,5554,378,9
822980,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/23/2019,13:00:00,REGULAR,5554,378,13
822981,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/23/2019,17:00:00,REGULAR,5554,378,17
822982,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/23/2019,21:00:00,REGULAR,5554,378,21
822983,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/24/2019,01:00:00,REGULAR,5554,378,1
822984,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/24/2019,05:00:00,REGULAR,5554,378,5
822985,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/24/2019,09:00:00,REGULAR,5554,378,9
822986,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/24/2019,13:00:00,REGULAR,5554,378,13
822987,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/24/2019,17:00:00,REGULAR,5554,378,17
822988,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/24/2019,21:00:00,REGULAR,5554,378,21


In [68]:
mta1["entries_diff"] = mta1.groupby(by = ["STATION","LINENAME","DIVISION","C/A","UNIT","SCP"]).ENTRIES.diff()
mta1["exits_diff"] = mta1.groupby(by = ["STATION","LINENAME","DIVISION","C/A","UNIT","SCP"]).EXITS.diff()
mta1["entries-exits"] = mta1["entries_diff"] - mta1["exits_diff"]

In [80]:
mta1.tail(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,time_hour,entries_diff,exits_diff,entries-exits
822979,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/23/2019,09:00:00,REGULAR,5554,378,9,0.0,0.0,0.0
822980,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/23/2019,13:00:00,REGULAR,5554,378,13,0.0,0.0,0.0
822981,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/23/2019,17:00:00,REGULAR,5554,378,17,0.0,0.0,0.0
822982,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/23/2019,21:00:00,REGULAR,5554,378,21,0.0,0.0,0.0
822983,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/24/2019,01:00:00,REGULAR,5554,378,1,0.0,0.0,0.0
822984,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/24/2019,05:00:00,REGULAR,5554,378,5,0.0,0.0,0.0
822985,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/24/2019,09:00:00,REGULAR,5554,378,9,0.0,0.0,0.0
822986,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/24/2019,13:00:00,REGULAR,5554,378,13,0.0,0.0,0.0
822987,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/24/2019,17:00:00,REGULAR,5554,378,17,0.0,0.0,0.0
822988,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,05/24/2019,21:00:00,REGULAR,5554,378,21,0.0,0.0,0.0


In [69]:
mta1.describe()

,ENTRIES,EXITS,time_hour,entries_diff,exits_diff,entries-exits
count,8.229890e+05,8.229890e+05,822989.000000,8.181190e+05,8.181190e+05,8.181190e+05
mean,4.137444e+07,3.392040e+07,10.537974,6.501273e+03,4.389621e+03,2.111652e+03
std,2.110233e+08,1.945992e+08,6.813749,3.880945e+06,2.153482e+06,2.952274e+06
min,0.000000e+00,0.000000e+00,0.000000,-1.437242e+09,-3.843496e+08,-1.086304e+09
25%,3.552810e+05,1.453550e+05,4.000000,9.000000e+00,8.000000e+00,-2.600000e+01
50%,2.179688e+06,1.249356e+06,10.000000,7.100000e+01,5.200000e+01,1.000000e+00
75%,6.796341e+06,4.610048e+06,16.000000,2.390000e+02,1.670000e+02,9.500000e+01
max,2.129343e+09,2.124127e+09,23.000000,2.055526e+09,1.078346e+09,1.888396e+09


In [70]:
mta2 = mta1.groupby(["STATION","LINENAME","DIVISION","C/A","UNIT","SCP","DATE"])[["ENTRIES","EXITS"]].sum()
mta2.head(40)

In [75]:
mta3 = mta1.groupby(["STATION","LINENAME"])[["ENTRIES","EXITS"]].sum().sort_values("ENTRIES",ascending = False)
mta3.head(40)

,,ENTRIES,EXITS
STATION,LINENAME,,
42 ST-PORT AUTH,ACENQRS1237W,1325100766251,1087383399661
TIMES SQ-42 ST,1237ACENQRSW,1050107602568,572042830342
3 AV-149 ST,25,871050714699,573974218369
34 ST-HERALD SQ,BDFMNQRW,792675667461,993997895389
183 ST,4,688797769222,440331418268
72 ST,123,595692069534,794986496814
FULTON ST,ACJZ2345,566891838239,371440694638
ELMHURST AV,MR,542939140010,785995739092
104 ST,JZ,530523541168,426588183018


In [77]:
mta1.sort_values("entries_diff")

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,time_hour,entries_diff,exits_diff,entries-exits
582980,R288,R275,00-00-03,183 ST,4,IRT,05/15/2019,12:00:00,REGULAR,140396510,51950488,12,-1.437242e+09,-350938302.0,-1.086304e+09
72426,N196,R285,00-05-01,FAR ROCKAWAY,A,IND,05/03/2019,16:00:00,REGULAR,65536,2,16,-8.392517e+08,-167841839.0,-6.714099e+08
167363,R258,R132,00-03-00,125 ST,456,IRT,05/03/2019,13:00:00,REGULAR,68858058,68916176,13,-3.676523e+08,-384349631.0,1.669731e+07
553728,R160A,R164,00-00-02,66 ST-LINCOLN,1,IRT,05/11/2019,17:00:00,REGULAR,1058563,262707,17,-1.013321e+08,-50788219.0,-5.054392e+07
170314,R289,R119,00-05-01,FORDHAM RD,4,IRT,05/03/2019,16:00:00,REGULAR,458752,7,16,-1.002045e+08,-16777623.0,-8.342692e+07
544936,R121,R290,01-06-00,HOUSTON ST,1,IRT,05/11/2019,17:00:00,REGULAR,196625,30,17,-6.826523e+07,-17065778.0,-5.119945e+07
551480,R151,R033,00-00-05,TIMES SQ-42 ST,1237ACENQRSW,IRT,05/15/2019,16:00:00,REGULAR,146,258,16,-3.696648e+07,-89243892.0,5.227741e+07
655279,H041,R152,00-00-01,CANARSIE-ROCKAW,L,BMT,05/18/2019,13:00:00,REGULAR,191,169,13,-1.541868e+07,-7550836.0,-7.867844e+06
399044,R601A,R108,02-00-01,BOROUGH HALL,2345R,IRT,05/06/2019,00:00:00,RECOVR AUD,7,0,0,-1.365362e+07,-8665111.0,-4.988508e+06
163490,R245A,R051,01-00-01,59 ST,456NQRW,IRT,05/02/2019,20:00:00,REGULAR,115,23,20,-1.273900e+07,-6526790.0,-6.212212e+06


In [79]:
mta1.iloc[72420:72440]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,time_hour,entries_diff,exits_diff,entries-exits
72420,N196,R285,00-05-01,FAR ROCKAWAY,A,IND,05/02/2019,16:00:00,REGULAR,839317244,167841841,16,0.0,0.0,0.0
72421,N196,R285,00-05-01,FAR ROCKAWAY,A,IND,05/02/2019,20:00:00,REGULAR,839317244,167841841,20,0.0,0.0,0.0
72422,N196,R285,00-05-01,FAR ROCKAWAY,A,IND,05/03/2019,00:00:00,REGULAR,839317244,167841841,0,0.0,0.0,0.0
72423,N196,R285,00-05-01,FAR ROCKAWAY,A,IND,05/03/2019,04:00:00,REGULAR,839317244,167841841,4,0.0,0.0,0.0
72424,N196,R285,00-05-01,FAR ROCKAWAY,A,IND,05/03/2019,08:00:00,REGULAR,839317244,167841841,8,0.0,0.0,0.0
72425,N196,R285,00-05-01,FAR ROCKAWAY,A,IND,05/03/2019,12:00:00,REGULAR,839317244,167841841,12,0.0,0.0,0.0
72426,N196,R285,00-05-01,FAR ROCKAWAY,A,IND,05/03/2019,16:00:00,REGULAR,65536,2,16,-839251708.0,-167841839.0,-671409869.0
72427,N196,R285,00-05-01,FAR ROCKAWAY,A,IND,05/03/2019,20:00:00,REGULAR,65536,2,20,0.0,0.0,0.0
72428,N202,R315,00-00-00,155 ST,BD,IND,04/27/2019,01:00:00,REGULAR,8938117,8025191,1,NaN,NaN,NaN
72429,N202,R315,00-00-00,155 ST,BD,IND,04/27/2019,05:00:00,REGULAR,8938131,8025234,5,14.0,43.0,-29.0
